In [51]:
import xgboost as xgb

import numpy as np
import os
import pandas as pd

from sklearn.model_selection import train_test_split
import sklearn

In [52]:
folder_train = '../features_train_in_use'

features_train = pd.read_csv(os.path.join(folder_train, os.listdir(folder_train)[0]))
for i in range(1,len(os.listdir(folder_train))):
    f = os.path.join(folder_train,os.listdir(folder_train)[i])
    if os.path.isfile(f):
        features_train = pd.concat([features_train, pd.read_csv(f)], axis=1)


In [53]:
folder_test = '../features_test_in_use'

features_test = pd.read_csv(os.path.join(folder_test, os.listdir(folder_test)[0]))
for i in range(1,len(os.listdir(folder_test))):
    f = os.path.join(folder_test,os.listdir(folder_test)[i])
    if os.path.isfile(f):
        features_test = pd.concat([features_test, pd.read_csv(f)], axis=1)

In [54]:
target_train = pd.read_csv("../data/stores_train.csv")
target_train = target_train['revenue'].values

#log transform
target_train = np.log1p(target_train)

In [55]:
model = xgb.XGBRegressor(tree_method="gpu_hist", enable_categorical=True)

In [56]:
# TRAIN

for col in features_train.columns:
    if col.startswith("categorical"):
        print(col)
        features_train[col] = features_train[col].astype("category")

categorical_chain_name
categorical_mall_name
categorical_plaace
categorical_sales_channel_name
categorical_street


In [57]:
# TEST

for col in features_test.columns:
    if col.startswith("categorical"):
        print(col)
        features_test[col] = features_test[col].astype("category")

categorical_chain_name
categorical_mall_name
categorical_plaace
categorical_sales_channel_name
categorical_street


In [58]:
model.fit(features_train, target_train)
xg_predictions = model.predict(features_test)

#Undo log transform
xg_predictions = np.expm1(xg_predictions)
print(xg_predictions)

[5.559127  3.1874957 0.6265483 ... 7.1922436 5.0795064 3.3501048]


In [59]:
X_train, X_test, y_train, y_test = train_test_split(features_train, target_train, test_size=0.2, random_state=42)

model2 = xgb.XGBRegressor(tree_method="gpu_hist", enable_categorical=True)

#log transform
y_train = np.log1p(y_train)

model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)

y_pred = np.expm1(y_pred)

print("Score: ",sklearn.metrics.mean_squared_log_error(y_test, y_pred) ** 0.5)

Score:  0.33857665526530484
